<a href="https://colab.research.google.com/github/Ronilmuchandi/economics-of-remote-work-city-opportunity/blob/main/notebooks/03a_city_reference_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/uscities.xlsx")


In [ ]:
df = df[[
    "city",
    "state_id",
    "county_fips",
    "lat",
    "lng",
    "population",
    "density"
]]




In [ ]:
df["city"] = df["city"].str.lower().str.strip()
df["state_id"] = df["state_id"].str.upper().str.strip()


In [ ]:
df["county_fips"] = (
    df["county_fips"]
    .astype(str)
    .str.replace(".0", "", regex=False)
    .str.zfill(5)
)



In [ ]:
df.head(2)
df.isna().sum()


,0
city,0
state_id,0
county_fips,0
lat,0
lng,0
population,0
density,0


In [ ]:
df.to_csv("/content/drive/MyDrive/city_reference_clean.csv", index=False)



In [ ]:
# ============================================================
# STEP 3.2 — ZILLOW CLEAN (FIXED VERSION)
# Fixes: excludes non-date columns from melt
# ============================================================

import pandas as pd

# ---------- 1. LOAD ----------
zillow = pd.read_csv("/content/drive/MyDrive/zillow.csv")

# ---------- 2. INSPECT ----------
print("Original shape:", zillow.shape)
print("Columns:")
print(zillow.columns.tolist())
display(zillow.head(2))

# ---------- 3. KEEP ONLY REQUIRED COLUMNS ----------
# Geography columns
geo_cols = ["RegionName", "StateName"]

# Date columns = those that look like YYYY-MM-DD
date_cols = [c for c in zillow.columns if c[:4].isdigit()]

# Subset dataset
zillow = zillow[geo_cols + date_cols]

# Rename for consistency
zillow = zillow.rename(columns={
    "RegionName": "city",
    "StateName": "state"
})

# Clean text
zillow["city"] = zillow["city"].str.lower().str.strip()
zillow["state"] = zillow["state"].str.upper().str.strip()

# ---------- 4. WIDE → LONG ----------
zillow_long = zillow.melt(
    id_vars=["city", "state"],
    value_vars=date_cols,
    var_name="date",
    value_name="zori"
)

# Convert date safely
zillow_long["date"] = pd.to_datetime(zillow_long["date"], format="%Y-%m-%d")

# Drop missing rent values
zillow_long = zillow_long.dropna(subset=["zori"])

# ---------- 5. FINAL CHECK ----------
print("Cleaned Zillow shape:", zillow_long.shape)
display(zillow_long.head(2))

# ---------- 6. SAVE ----------
zillow_long.to_csv(
    "/content/drive/MyDrive/zillow_clean_long.csv",
    index=False
)

print("Saved: zillow_clean_long.csv")


Original shape: (692, 136)
Columns:
['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2

,RegionID,SizeRank,RegionName,RegionType,StateName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,...,2025-02-28,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30
0,102001,0,United States,country,NaN,1154.552289,1160.825274,1169.472351,1178.330912,1187.105490,...,1898.645733,1911.434221,1921.291874,1928.586785,1933.384822,1935.477235,1935.901356,1934.381605,1930.806380,1925.281186
1,394913,1,"New York, NY",msa,NY,2180.969661,2195.187121,2213.489936,2232.223598,2246.638993,...,3163.366748,3192.406481,3222.475993,3249.390907,3280.298984,3309.101957,3329.595797,3326.866766,3314.129267,3293.660323


Cleaned Zillow shape: (47811, 4)


,city,state,date,zori
0,united states,NaN,2015-01-31,1154.552289
1,"new york, ny",NY,2015-01-31,2180.969661


Saved: zillow_clean_long.csv


In [ ]:
# ============================================================
# STEP 3.3 — POSTINGS CLEAN + AGGREGATE (ONE-CELL, FIXED)
# Input : postings.csv (Google Drive root)
# Output: postings_city_month.csv
# ============================================================

import pandas as pd

# ---------- LOAD ----------
posts = pd.read_csv(
    "/content/drive/MyDrive/postings.csv",
    usecols=["location", "remote_allowed", "listed_time"],
    low_memory=False
)

print("Raw shape:", posts.shape)
display(posts.head(2))

# ---------- CLEAN ----------
# Fix remote flag (NaN = not remote)
posts["remote_allowed"] = posts["remote_allowed"].fillna(0).astype(int)

# Convert timestamp (milliseconds → datetime)
posts["listed_time"] = pd.to_datetime(
    posts["listed_time"], unit="ms", errors="coerce"
)
posts = posts.dropna(subset=["listed_time"])

# Create month bucket
posts["month"] = posts["listed_time"].dt.to_period("M").astype(str)

# ---------- PARSE CITY + STATE ----------
def parse_city_state(x):
    if pd.isna(x):
        return pd.Series([None, None])
    parts = [p.strip() for p in str(x).split(",")]
    if len(parts) >= 2:
        return pd.Series([parts[0].lower(), parts[1].upper()])
    return pd.Series([parts[0].lower(), None])

posts[["city", "state"]] = posts["location"].apply(parse_city_state)
posts = posts.dropna(subset=["city"])

# ---------- AGGREGATE ----------
agg = (
    posts
    .groupby(["city", "state", "month"], as_index=False)
    .agg(
        total_postings=("remote_allowed", "size"),
        remote_postings=("remote_allowed", "sum")
    )
)

print("Aggregated shape:", agg.shape)
display(agg.head(2))

# ---------- SAVE ----------
agg.to_csv(
    "/content/drive/MyDrive/postings_city_month.csv",
    index=False
)

print("Saved: postings_city_month.csv")


Raw shape: (123849, 3)


,location,remote_allowed,listed_time
0,"Princeton, NJ",NaN,1.713398e+12
1,"Fort Collins, CO",NaN,1.712858e+12


Aggregated shape: (8348, 5)


,city,state,month,total_postings,remote_postings
0,abbeville,LA,2024-04,5,0
1,abbeville,SC,2024-04,1,0


Saved: postings_city_month.csv


In [15]:
# ============================================================
# STEP 4 — FINAL CORRECT MERGE (ZILLOW CITY FIX)
# ============================================================

import pandas as pd

# ---------- LOAD ----------
cities = pd.read_csv("/content/drive/MyDrive/city_reference_clean.csv")
zillow = pd.read_csv("/content/drive/MyDrive/zillow_clean_long.csv")
posts  = pd.read_csv("/content/drive/MyDrive/postings_city_month.csv")

# ---------- STANDARDIZE POSTS ----------
posts["city"]  = posts["city"].str.lower().str.strip()
posts["state"] = posts["state"].str.upper().str.strip()
posts["month"] = pd.to_datetime(posts["month"]).dt.to_period("M").astype(str)

# ---------- STANDARDIZE ZILLOW ----------
# Zillow city is like "new york, ny" → extract city only
zillow["city"] = (
    zillow["city"]
    .str.lower()
    .str.split(",")
    .str[0]
    .str.strip()
)

zillow["state"] = zillow["state"].str.upper().str.strip()
zillow["month"] = pd.to_datetime(zillow["date"]).dt.to_period("M").astype(str)
zillow = zillow.drop(columns=["date"])

# ---------- FILTER TO CITIES ZILLOW ACTUALLY HAS ----------
zillow_cities = zillow[["city", "state"]].drop_duplicates()

posts = posts.merge(
    zillow_cities,
    on=["city", "state"],
    how="inner"
)

# ---------- MERGE POSTS + ZILLOW ----------
panel = posts.merge(
    zillow,
    on=["city", "state", "month"],
    how="left"
)

# ---------- ADD CITY METADATA ----------
cities["city"] = cities["city"].str.lower().str.strip()
cities["state_id"] = cities["state_id"].str.upper().str.strip()

panel = panel.merge(
    cities,
    left_on=["city", "state"],
    right_on=["city", "state_id"],
    how="left"
).drop(columns=["state_id"])

# ---------- FINAL CHECK ----------
print("Master panel shape:", panel.shape)
print("Non-null zori count:", panel['zori'].notna().sum())
display(panel.head(3))

# ---------- SAVE ----------
panel.to_csv("/content/drive/MyDrive/master_city_month.csv", index=False)
print("Saved: master_city_month.csv")


Master panel shape: (655, 11)
Non-null zori count: 508


,city,state,month,total_postings,remote_postings,zori,county_fips,lat,lng,population,density
0,aberdeen,SD,2024-04,3,0,NaN,46013.0,45.4649,-98.4686,28297.0,659.2
1,aberdeen,WA,2024-04,8,0,1387.180171,53027.0,46.9757,-123.8094,17040.0,605.1
2,abilene,TX,2024-04,60,3,1300.080222,48441.0,32.4543,-99.7384,127130.0,460.2


Saved: master_city_month.csv


In [ ]:
import pandas as pd

pd.read_csv("/content/drive/MyDrive/master_city_month.csv").head()


,city,state,month,total_postings,remote_postings,zori,county_fips,lat,lng,population,density
0,abbeville,LA,2024-04,5,0,NaN,22113.0,29.9751,-92.1265,11086.0,707.9
1,abbeville,SC,2024-04,1,0,NaN,45001.0,34.1787,-82.3774,4863.0,302.1
2,abbotsford,WI,2024-04,1,0,NaN,55073.0,44.9435,-90.3174,2100.0,273.6
3,abbott,TX,2024-04,2,1,NaN,48217.0,31.8868,-97.0799,323.0,210.7
4,aberdeen,MD,2024-04,7,0,NaN,24025.0,39.5151,-76.1733,16831.0,933.5


In [14]:
# ================================
# CHECK DATASETS: COLUMNS + SAMPLES
# ================================

import pandas as pd

# Reload cleanly (important)
cities = pd.read_csv("/content/drive/MyDrive/city_reference_clean.csv")
zillow = pd.read_csv("/content/drive/MyDrive/zillow_clean_long.csv")
posts  = pd.read_csv("/content/drive/MyDrive/postings_city_month.csv")

print("\n===== ZILLOW =====")
print(zillow.columns)
display(zillow.head(5))

print("\n===== POSTINGS =====")
print(posts.columns)
display(posts.head(5))

print("\n===== CITY REFERENCE =====")
print(cities.columns)
display(cities.head(5))



===== ZILLOW =====
Index(['city', 'state', 'date', 'zori'], dtype='object')


,city,state,date,zori
0,united states,NaN,2015-01-31,1154.552289
1,"new york, ny",NY,2015-01-31,2180.969661
2,"los angeles, ca",CA,2015-01-31,1754.825391
3,"chicago, il",IL,2015-01-31,1326.149779
4,"dallas, tx",TX,2015-01-31,1060.295251



===== POSTINGS =====
Index(['city', 'state', 'month', 'total_postings', 'remote_postings'], dtype='object')


,city,state,month,total_postings,remote_postings
0,abbeville,LA,2024-04,5,0
1,abbeville,SC,2024-04,1,0
2,abbotsford,WI,2024-04,1,0
3,abbott,TX,2024-04,2,1
4,aberdeen,MD,2024-04,7,0



===== CITY REFERENCE =====
Index(['city', 'state_id', 'county_fips', 'lat', 'lng', 'population',
       'density'],
      dtype='object')


,city,state_id,county_fips,lat,lng,population,density
0,new york,NY,36081,40.6943,-73.9249,18832416,10943.7
1,los angeles,CA,6037,34.1141,-118.4068,11885717,3165.7
2,chicago,IL,17031,41.8375,-87.6866,8489066,4590.3
3,miami,FL,12086,25.7840,-80.2101,6113982,4791.1
4,houston,TX,48201,29.7860,-95.3885,6046392,1386.2
